# O que tem aqui

Parecido com o notebook-2, mas fazendo com loop for ao invés de tentar vetorizar tudo.

# Imports

In [2]:
import functools
import itertools
import os
import re
from collections import defaultdict, namedtuple
from pathlib import Path
import toolz
from dataclasses import dataclass

import more_itertools

# Data preprocessing

## Download dos dados

Para fazer download dos dados, vá até a pasta raiz e digite:
```bash
make get_raw_data

```
Para gerar esse Makefile, eu dei a seguinte query ao chat GPT:
```
write a makefile to do the following:
- download file from http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz
- unzip it to the folder data/raw
- it should be on step "get_raw_data"
- operation should not be repeated if files already exist
- write help for the steps
```


# Conversão dos dados para um formato mais fácil de trabalhar

In [3]:
RAW_DATA_BASEDIR = Path('../data/raw')
!ls -lht {RAW_DATA_BASEDIR}

raw_files_path = {
    'query': RAW_DATA_BASEDIR / 'CISI.QRY',
    'all': RAW_DATA_BASEDIR / 'CISI.ALL',
    'rel': RAW_DATA_BASEDIR / 'CISI.REL',
}


total 6200
-rw-r--r--  1 marcospiau  staff   757K Feb 22 19:27 cisi.tar.gz
-rw-r--r--  1 marcospiau  staff    79K Feb 28  1994 CISI.REL
-rw-r--r--  1 marcospiau  staff    67K Feb 28  1994 CISI.QRY
-rw-r--r--  1 marcospiau  staff   4.5K Feb 28  1994 CISI.BLN
-rw-r--r--  1 marcospiau  staff   2.1M Feb 28  1994 CISI.ALL


Arquivos QRY e ALL são mais complicados e precisam de processamento. Tentei deixar processamento semelhante pra conseguir fazer os dois casos com uma mesma funcao.

Usei bastante o chatgpt, principalmente para escrever docstrings e typehints. Uma coisa curiosa é que constantemente ele removia a dataclass e utilizava classes padrão, de forma que precisei constamente instruir ele a a manter as dataclasses.

Utilizei bastante a bibliioteca toolz para deixar o código com uma abordagem mais funcional.

In [4]:
# (.I) ID
# (.T) Title
# (.W) Abstract
# (.B) Publication date of the article
# (.A) Author list
# (.N) Information when entry was added
# (.X) List of cross-references to other documents

renames_docs_all = {
    '.I': 'id',
    '.T': 'title',
    '.W': 'abstract',
    '.B': 'publication_date',
    '.A': 'author_list',
    '.N': 'added_date',
    '.X': 'cross_references'
}

process_doc_all = toolz.compose_left(
    # for each tag, get renamed tag, and join texts for tag
    functools.partial(more_itertools.map_reduce,
                      keyfunc=lambda x: renames_docs_all[x.tag],
                      valuefunc=lambda x: x.text,
                      reducefunc=lambda x: ' '.join(x)),
    # keeps only desired keys
    toolz.curried.keyfilter(lambda x: x in {'id', 'title', 'abstract'}),
    # convert id to int
    toolz.curried.update_in(keys=['id'], func=int)
)

# (.I) ID
# (.W) Query
# (.A) Author list
# (.N) Authors name and some keywords on what the query searches for

renames_docs_queries = {
    '.I': 'id',
    '.W': 'query',
    '.T': 'title',
    '.A': 'author_list',
    # '.N': 'other_query_infos',
    '.B': 'publication_date',
    # '.X': 'cross_references'
}

process_doc_qry = toolz.compose_left(
    functools.partial(more_itertools.map_reduce,
                      keyfunc=lambda x: renames_docs_queries[x.tag],
                      valuefunc=lambda x: x.text,
                      reducefunc=lambda x: ' '.join(x)),
    toolz.curried.keyfilter(lambda x: x in {'id', 'query'}),
    toolz.curried.update_in(keys=['id'], func=int))

In [5]:
import itertools
import re
from typing import Any, Callable, Dict, Iterable, List, Tuple, Union


@dataclass
class IdTagText:
    id: str
    tag: str
    text: str


def parse_cisi_all_or_qry(
    path: str,
    process_doc_fn: Callable[[Iterable[IdTagText]], Dict[str, Any]],
    return_dict: bool = False,
) -> Union[List[Dict[str, Any]], Dict[str, Any]]:
    """Parses a CISI all or query file.

    Args:
        path: A string representing the path to the file to parse.
        process_doc_fn: A function that receives a list of IdTagText or an iterable of
          IdTagText as input, and returns a dictionary.
        return_dict: A boolean indicating whether to return a dictionary or a list of
          dictionaries.

    Returns:
        If return_dict is True, a dictionary with the document IDs as keys and
        processed documents as values. Otherwise, a list of processed documents.

    Raises:
        AssertionError: If the document IDs are not unique.
    """
    markers = [r'^\.I\s(\d+)$', '\.T$', '\.A$', '\.B$', '\.W$', '\.X$', '\.N$']
    marker_pattern = re.compile('|'.join(markers))

    def gen_items() -> Iterable[IdTagText]:
        with open(path, 'r') as f:
            for line in map(str.strip, f):
                match = marker_pattern.match(line)
                # match occurs for lines with tags
                if match:
                    # if match.group(1) is not None, it means that the tag is .I
                    # and the group contains the ID
                    if match.group(1):
                        id_ = match.group(1)
                        yield IdTagText(id_, '.I', id_)
                    else:
                        tag = match.group(0).strip()
                else:
                    # if match is None, it means that the line is a text
                    # just propagate tag and id
                    yield IdTagText(id_, tag, line)

    out = [
        process_doc_fn(group)
        for _, group in itertools.groupby(gen_items(), key=lambda x: x.id)
    ]
    assert len(out) == len({x['id'] for x in out}), 'IDs are not unique'
    return {x['id']: x for x in out} if return_dict else out


process_cisi_all = functools.partial(parse_cisi_all_or_qry,
                                     process_doc_fn=process_doc_all)
process_cisi_qry = functools.partial(parse_cisi_all_or_qry,
                                     process_doc_fn=process_doc_qry)

In [6]:
corpus = process_cisi_all(raw_files_path['all'])
corpus[0]

{'id': 1,
 'title': '18 Editions of the Dewey Decimal Classifications',
 'abstract': "The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad."}

In [7]:
queries = process_cisi_qry(raw_files_path['query'])
len(queries), queries[0]

(112,
 {'id': 1,
  'query': 'What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?'})

O arquivo qrels é mais simples e pode ser lido de forma mais simples. O resultado é um dicionário com a query como chave e uma lista de documentos relevantes para aquela query.

In [8]:
def load_qrels(path):
    out = defaultdict(set)
    with open(path, 'r') as f:
        for line in map(str.strip, f):
            qid, docid, _, _ = line.split()
            out[int(qid)].add(int(docid))
    out.default_factory = None

    return out

qrels = load_qrels(raw_files_path['rel'])
list(itertools.islice(qrels.items(), 5))

[(1,
  {28,
   35,
   38,
   42,
   43,
   52,
   65,
   76,
   86,
   150,
   189,
   192,
   193,
   195,
   215,
   269,
   291,
   320,
   429,
   465,
   466,
   482,
   483,
   510,
   524,
   541,
   576,
   582,
   589,
   603,
   650,
   680,
   711,
   722,
   726,
   783,
   813,
   820,
   868,
   869,
   894,
   1162,
   1164,
   1195,
   1196,
   1281}),
 (2,
  {29,
   68,
   197,
   213,
   214,
   309,
   319,
   324,
   429,
   499,
   636,
   669,
   670,
   674,
   690,
   692,
   695,
   700,
   704,
   709,
   720,
   731,
   733,
   738,
   740,
   1136}),
 (3,
  {60,
   85,
   114,
   123,
   126,
   131,
   133,
   136,
   138,
   140,
   346,
   359,
   363,
   372,
   412,
   445,
   454,
   461,
   463,
   469,
   532,
   537,
   540,
   553,
   554,
   555,
   585,
   590,
   599,
   640,
   660,
   664,
   803,
   901,
   909,
   911,
   1027,
   1053,
   1169,
   1179,
   1181,
   1190,
   1191,
   1326}),
 (4, {310, 315, 321, 329, 332, 420, 601, 980}),
 (

In [9]:
# Filtrando queries que aparecem no qrels:
queries = [x for x in queries if x['id'] in qrels.keys()]
len(queries), queries[0]

(76,
 {'id': 1,
  'query': 'What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?'})

Abaixo, vemos que todos os qrels possuem relevância igual a zero. Por isso, vamos interpretar que caso um documento esteja relacionado a uma query, ele é relevante para essa query.

In [10]:
import pandas as pd
df = pd.read_csv(raw_files_path['rel'], delim_whitespace=True, header=None, names=['qid', 'docid', 'nao', 'sei']).astype(str)
display(df.describe())
del df

,qid,docid,nao,sei
count,3114,3114,3114,3114
unique,76,1162,1,1
top,44,375,0,0.0
freq,155,15,3114,3114


# Classe BM25

Utilizei bastante o ChatGPT para desevolver essa classe, comecei com uma classe mais simples (loops for) e fui modificando para conseguir deixar um pouco mais vetorizado. Estamos usando o CountVectorizer para facilitar a tokenização dos documentos e queries, o que também ajuda pois estão incluídos os stop words.

Novamente, docstrings e typehints foram gerados pelo chatgpt.

Utilizamos numpy e matrizes esparsas scipy para facilitar vetorização e operações matriciais.

Valores default de k1 e b foram escolhidos pelo ChatGPT.

In [249]:
import re
from collections import defaultdict
from typing import List, Tuple


def tokenize(text: str) -> List[str]:
    """Tokenize the input text.

    Args:
        text: Input text to be tokenized.

    Returns:
        List of tokens in the text after cleaning.
    """
    # cleaned_text = re.sub(r'[^\w\s]', '', text).lower()
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
    return cleaned_text.split()


def build_inverted_index(ids_and_tokens: List[Tuple[int, List[str]]]) -> dict:
    """Build an inverted index from the given document ids and tokens.

    Args:
        ids_and_tokens: List of document ids and corresponding list of tokens.

    Returns:
        Inverted index where keys are tokens and values are sets of document ids
        that contain the token.
    """
    inverted_index = defaultdict(set)
    for doc_id, tokens in ids_and_tokens:
        for token in tokens:
            inverted_index[token].add(doc_id)
    inverted_index.default_factory = None
    return inverted_index


# inverted_index = build_inverted_index((x['id'] for x in corpus[:5]), (tokenize(x['abstract']) for x in corpus[:5]))
inverted_index = build_inverted_index(
    (x['id'], tokenize(x['abstract'])) for x in corpus[:5])
inverted_index


defaultdict(None,
            {'the': {1, 2, 3, 4, 5},
             'present': {1},
             'study': {1},
             'is': {1, 2, 3, 5},
             'a': {1, 2, 3, 4, 5},
             'history': {1},
             'of': {1, 2, 3, 4, 5},
             'dewey': {1},
             'decimal': {1},
             'classification': {1},
             'first': {1, 5},
             'edition': {1},
             'ddc': {1},
             'was': {1, 5},
             'published': {1},
             'in': {1, 2, 3, 4, 5},
             '1876': {1},
             'eighteenth': {1},
             '1971': {1},
             'and': {1, 2, 3, 4, 5},
             'future': {1},
             'editions': {1},
             'will': {1, 3},
             'continue': {1},
             'to': {1, 2, 3, 5},
             'appear': {1},
             'as': {1, 2, 3, 5},
             'needed': {1},
             'spite': {1},
             'ddcs': {1},
             'long': {1},
             'healthy': {1},
             'lif

In [285]:
from typing import Dict, Set
import math


def calculate_idf(inverted_index: Dict[str, Set[int]], corpus_size: int,
                  eps: float = 1e-8) -> Dict[str, float]:
    """Calculate inverse document frequency (IDF) for each token in a corpus.

    Args:
        inverted_index: A dictionary where keys are tokens and values are sets of
            document ids that contain the token.
        corpus_size: The total number of documents in the corpus.
        eps: A small value added to the denominator to avoid division by zero.

    Returns:
        A dictionary where keys are tokens and values are IDF scores.
    """
    idf = {}
    for token, doc_ids in inverted_index.items():
        idf[token] = math.log(
            (corpus_size - len(doc_ids) + 0.5) / (len(doc_ids) + 0.5) + eps)
    return idf


calculate_idf(inverted_index, 10)

{'the': 9.999999889225291e-09,
 'present': 1.8458266920772781,
 'study': 1.8458266920772781,
 'is': 0.3677247870483942,
 'a': 9.999999889225291e-09,
 'history': 1.8458266920772781,
 'of': 9.999999889225291e-09,
 'dewey': 1.8458266920772781,
 'decimal': 1.8458266920772781,
 'classification': 1.8458266920772781,
 'first': 1.223775434563292,
 'edition': 1.8458266920772781,
 'ddc': 1.8458266920772781,
 'was': 1.223775434563292,
 'published': 1.8458266920772781,
 'in': 9.999999889225291e-09,
 '1876': 1.8458266920772781,
 'eighteenth': 1.8458266920772781,
 '1971': 1.8458266920772781,
 'and': 9.999999889225291e-09,
 'future': 1.8458266920772781,
 'editions': 1.8458266920772781,
 'will': 1.223775434563292,
 'continue': 1.8458266920772781,
 'to': 0.3677247870483942,
 'appear': 1.8458266920772781,
 'as': 0.3677247870483942,
 'needed': 1.8458266920772781,
 'spite': 1.8458266920772781,
 'ddcs': 1.8458266920772781,
 'long': 1.8458266920772781,
 'healthy': 1.8458266920772781,
 'life': 1.845826692077

In [294]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from tqdm.auto import tqdm
import math
from collections import Counter
import heapq


class BM25:

    def __init__(self, corpus: List[dict], k1: float = 1.2, b: float = 0.75):
        self.k1 = k1
        self.b = b

        # corpus is tokenized
        self.corpus = {doc['id']: tokenize(doc['text']) for doc in corpus}
        # remove english stopwords from tokens
        # use sklearn set
        stop_words = set(CountVectorizer(stop_words='english').get_stop_words())
        self.corpus = {
            id_: [token for token in tokens if token not in stop_words]
            for id_, tokens in self.corpus.items()
        }
        self.inverted_index = build_inverted_index(self.corpus.items())
        self.idf = calculate_idf(self.inverted_index, len(corpus))
        self.doc_lengths = {
            id_: len(text)
            for id_, text in self.corpus.items()
        }
        self.avg_doc_length = sum(self.doc_lengths.values()) / len(
            self.doc_lengths)

    def score(self, query: str, doc_id: int) -> float:
        """Calculate the relevance score of a document for a given query.

        Args:
            query: Query text.
            doc_id: ID of the document in the corpus.

        Returns:
            Relevance score of the document for the query.
        """
        query_tokens = tokenize(query)
        doc_token_counts = Counter(self.corpus[doc_id])
        score = 0
        for token in query_tokens:
            if token not in self.idf:
                continue
            tf = doc_token_counts[token]
            score += self.idf[token] * tf * (self.k1 + 1) / (
                tf + self.k1 *
                (1 - self.b +
                 self.b * self.doc_lengths[doc_id] / self.avg_doc_length))
        return score

    def retrieve(self, query: str, k: int = 10) -> List[dict]:
        """Retrieve the top-k documents for a given query.

        Args:
            query: Query text.
            k: Number of documents to retrieve.

        Returns:
            List of document dictionaries with 'id' and 'score' keys.
        """
        ids_and_scores = ((doc_id, self.score(query, doc_id))
                          for doc_id in self.corpus.keys())
        top_ids_and_scores = heapq.nlargest(k,
                                            ids_and_scores,
                                            key=lambda x: x[1])
        return [{
            'id': docid,
            'score': score
        } for  docid, score in top_ids_and_scores]

    def get_results_for_all_queries(self,
                                    queries: List[dict],
                                    k: int = 10) -> Dict[int, List[dict]]:
        """Retrieve the top-k documents for all queries.

        Args:
            queries: List of query dictionaries with 'id' and 'text' keys.
            k: Number of documents to retrieve.

        Returns:
            Dictionary mapping query IDs to a list of document dictionaries with 'id' and 'score' keys.
        """
        return {
            query['id']: self.retrieve(query['query'], k)
            for query in tqdm(queries, desc='Retrieving')
        }

Exemplo de uso:

In [295]:
corpus_abstracts = [{'id': x['id'], 'text': x['abstract']} for x in corpus]
bm25 = BM25(corpus=corpus_abstracts)
# bm25 = BM25(corpus=corpus_abstracts, k1=0.9, b = 0.4)

In [296]:
bm25.score(query=corpus[0]['abstract'], doc_id=1)

201.6932431020489

In [297]:
corpus[0]

{'id': 1,
 'title': '18 Editions of the Dewey Decimal Classifications',
 'abstract': "The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad."}

In [298]:
bm25.retrieve(query=corpus[0]['abstract'], k=10)

[{'id': 1, 'score': 201.6932431020489},
 {'id': 1152, 'score': 33.71480550367359},
 {'id': 20, 'score': 31.67684285115838},
 {'id': 260, 'score': 31.25732765173878},
 {'id': 354, 'score': 30.366258625359094},
 {'id': 361, 'score': 27.98861285737742},
 {'id': 1251, 'score': 27.160504746047373},
 {'id': 414, 'score': 25.544575443634066},
 {'id': 1393, 'score': 25.30211193094718},
 {'id': 1442, 'score': 22.671471892287038}]

In [299]:
bm25.retrieve(query=corpus[0]['abstract'], k=10)

[{'id': 1, 'score': 201.6932431020489},
 {'id': 1152, 'score': 33.71480550367359},
 {'id': 20, 'score': 31.67684285115838},
 {'id': 260, 'score': 31.25732765173878},
 {'id': 354, 'score': 30.366258625359094},
 {'id': 361, 'score': 27.98861285737742},
 {'id': 1251, 'score': 27.160504746047373},
 {'id': 414, 'score': 25.544575443634066},
 {'id': 1393, 'score': 25.30211193094718},
 {'id': 1442, 'score': 22.671471892287038}]

In [300]:
bm25.retrieve(query=queries[0]['query'], k=10)

[{'id': 722, 'score': 24.59373264685542},
 {'id': 1299, 'score': 22.213908816766562},
 {'id': 759, 'score': 20.107476664096144},
 {'id': 429, 'score': 18.868289966659894},
 {'id': 1281, 'score': 18.446223473995317},
 {'id': 1055, 'score': 18.38260908887326},
 {'id': 76, 'score': 17.080375848971116},
 {'id': 65, 'score': 16.196460758066124},
 {'id': 813, 'score': 16.039765631097616},
 {'id': 589, 'score': 15.721055731869683}]

In [324]:
results_top10 = bm25.get_results_for_all_queries(queries, k=100000)

Retrieving: 100%|██████████| 76/76 [00:02<00:00, 28.81it/s]


# Avaliacao das métricas

Para facilitar avaliacão das métricas e ter um script confiável, vamos utilizar o script trec_eval. Ele está instalado na pasta `bin`, dentro da raiz do projeto.

## Convertendo dados para format trec_eval

## QRELS

In [325]:
from typing import Set

def convert_qrels_to_trec_format(qrels: Dict[int, Set[int]], output_path: str):
    """Convert qrels to TREC format.

    Args:
        qrels: Dictionary of qrels.
        output_path: Path to the output file.
    """
    with open(output_path, 'w') as f:
        for qid, docids in qrels.items():
            for docid in docids:
                f.write(f'{qid} 0 {docid} 1\n')

!mkdir -pv '../data/processed/cisi'
convert_qrels_to_trec_format(qrels, '../data/processed/cisi/qrels.txt')
!head '../data/processed/cisi/qrels.txt'

1 0 1281 1
1 0 650 1
1 0 1162 1
1 0 524 1
1 0 269 1
1 0 1164 1
1 0 783 1
1 0 894 1
1 0 150 1
1 0 28 1


## Results

In [326]:
def convert_results_to_trec_eval_format(results: Dict[str, List[str]], output_path: str):
    """Converts results to trec_eval format.

    Args:
        results: Dictionary of results.
        output_path: Path to the output file.
    """
    with open(output_path, 'w') as f:
        for query_id, docs in results.items():
            for rank, doc in enumerate(docs, start=1):
                doc_id = doc['id']
                score = doc['score']
                line = f"{query_id:} Q0 {doc_id} {rank} {score} RUN\n"
                f.write(line)

convert_results_to_trec_eval_format(results_top10, '../data/processed/cisi/results_abstract_top10.txt')
!head '../data/processed/cisi/results_abstract_top10.txt'

1 Q0 722 1 24.59373264685542 RUN
1 Q0 1299 2 22.213908816766562 RUN
1 Q0 759 3 20.107476664096144 RUN
1 Q0 429 4 18.868289966659894 RUN
1 Q0 1281 5 18.446223473995317 RUN
1 Q0 1055 6 18.38260908887326 RUN
1 Q0 76 7 17.080375848971116 RUN
1 Q0 65 8 16.196460758066124 RUN
1 Q0 813 9 16.039765631097616 RUN
1 Q0 589 10 15.721055731869683 RUN


In [328]:
!../bin/trec_eval-9.0.7/trec_eval ../data/processed/cisi/qrels.txt ../data/processed/cisi/results_abstract_top10.txt -M 100 -m P.1,3,5 # -m all_trec

P_1                   	all	0.4868
P_3                   	all	0.4035
P_5                   	all	0.3842


In [338]:
!../bin/trec_eval-9.0.7/trec_eval ../data/processed/cisi/qrels.txt ../data/processed/cisi/results_abstract_top10.txt -M 100 -m P.1,3,5 #-m all_trec

P_1                   	all	0.4868
P_3                   	all	0.4035
P_5                   	all	0.3842


In [284]:
!../bin/trec_eval-9.0.7/trec_eval ../data/processed/cisi/qrels.txt ../data/processed/cisi/results_abstract_top10.txt -M 100 -m all_trec

runid                 	all	RUN
num_q                 	all	76
num_ret               	all	7600
num_rel               	all	3114
num_rel_ret           	all	1014
map                   	all	0.1361
gm_map                	all	0.0823
Rprec                 	all	0.2011
bpref                 	all	0.4116
recip_rank            	all	0.5687
iprec_at_recall_0.00  	all	0.6186
iprec_at_recall_0.10  	all	0.3971
iprec_at_recall_0.20  	all	0.2646
iprec_at_recall_0.30  	all	0.1791
iprec_at_recall_0.40  	all	0.1189
iprec_at_recall_0.50  	all	0.0698
iprec_at_recall_0.60  	all	0.0397
iprec_at_recall_0.70  	all	0.0240
iprec_at_recall_0.80  	all	0.0184
iprec_at_recall_0.90  	all	0.0063
iprec_at_recall_1.00  	all	0.0063
P_5                   	all	0.3605
P_10                  	all	0.3039
P_15                  	all	0.2588
P_20                  	all	0.2401
P_30                  	all	0.2079
P_100                 	all	0.1334
P_200                 	all	0.0667
P_500                 	all	0.0267
P_1000                	all	

In [241]:
!../bin/trec_eval-9.0.7/trec_eval ../data/processed/cisi/qrels.txt ../data/processed/cisi/results_abstract_top10.txt -J

runid                 	all	RUN
num_q                 	all	76
num_ret               	all	2661
num_rel               	all	3114
num_rel_ret           	all	2661
map                   	all	0.8678
gm_map                	all	0.8629
Rprec                 	all	0.8678
bpref                 	all	0.8678
recip_rank            	all	1.0000
iprec_at_recall_0.00  	all	1.0000
iprec_at_recall_0.10  	all	1.0000
iprec_at_recall_0.20  	all	1.0000
iprec_at_recall_0.30  	all	1.0000
iprec_at_recall_0.40  	all	1.0000
iprec_at_recall_0.50  	all	1.0000
iprec_at_recall_0.60  	all	1.0000
iprec_at_recall_0.70  	all	0.9342
iprec_at_recall_0.80  	all	0.8158
iprec_at_recall_0.90  	all	0.4211
iprec_at_recall_1.00  	all	0.1447
P_5                   	all	0.9737
P_10                  	all	0.9342
P_15                  	all	0.8684
P_20                  	all	0.8059
P_30                  	all	0.7092
P_100                 	all	0.3397
P_200                 	all	0.1751
P_500                 	all	0.0700
P_1000                	all	

In [207]:
!../bin/trec_eval-9.0.7/trec_eval -h

trec_eval [-h] [-q] [-m measure[.params] [-c] [-n] [-l <num>]
   [-D debug_level] [-N <num>] [-M <num>] [-R rel_format] [-T results_format]
   rel_info_file  results_file 
 
Calculate and print various evaluation measures, evaluating the results  
in results_file against the relevance info in rel_info_file. 
 
There are a fair number of options, of which only the lower case options are 
normally ever used.   
 --help:
 -h: Print full help message and exit. Full help message will include
     descriptions for any measures designated by a '-m' parameter, and
     input file format descriptions for any rel_info_format given by '-R'
     and any top results_format given by '-T.'
     Thus to see all info about preference measures use
          trec_eval -h -m all_prefs -R prefs -T trec_results 
 --version:
 -v: Print version of trec_eval and exit.
 --query_eval_wanted:
 -q: In addition to summary evaluation, give evaluation for each query/topic
 --measure measure_name[.measure_params]:
 -m